# 01_data_processing.ipynb

In this script, we load all the datasets that we will use in the analysis script. This includes the following datasets:

1. PWMS used in the hackathon, initially downloaded from ATtRACT
2. Results obtained when re-running the hackathon scripts 
3. Supplementary data from three publications that show experimentally validated interactions between COVID-19 and RNA binding Proteins (RBPs)

TO-DO:
- get amino acid sequences of proteins, this will be the simplest features 
- another feature is whether RBP binds covid and maybe also logFC from protein abundance experiments
- can be one hot encoded? 
- but i think it might be able to learn based on amino acid sequences what RNA sequence motif it can bind to 
- use consense list of published COVID-RBPs as COVID-RBP positive label everything else is COVID-RBP negative label 
- predict motifs for RBPs with no PWM 
- once all proteins have a predicted RNA binding motif 
- can basically re-do hackathon analysis but on the COVID variants to see which ones get affected - how RBP activity changes 

## load all modules/packages

In [ ]:
try:
    import google.colab
    # Running on Google Colab, so install Biopython first
    !pip install biopython
    !pip install pyensembl
except ImportError:
    pass

     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 130 kB 8.7 MB/s 
  Created wheel for pyensembl: filename=pyensembl-1.9.4-py3-none-any.whl size=56443 sha256=d55c497f13d7d5ea41cf3963ce8372a56ed5b5025809b9d3b507cc8f33f755f8
  Stored in directory: /root/.cache/pip/wheels/14/e7/fe/08360ab53ff9bebb6daf956827c04e894d76dc0f64eb7502ef
  Created wheel for datacache: filename=datacache-1.1.5-py3-none-any.whl size=16473 sha256=8f765935055115c030e89c7f6a8dcb9a75449b3ff3a0fe407fad094fa105c22e
  Stored in directory: /root/.cache/pip/wheels/83/dd/38/d52c1ed3ad3ec655ea30a429de1cf89fac1009e8cb0365703a
  Created wheel for gtfparse: filename=gtfparse-1.2.1-py3-none-any.whl size=11637 sha256=2c66d2ce92a5fe1a7310e3e3511a2cbfc24dbb991e0be1fd3f581b8c88f316e4
  Stored in directory: /root/.cache/pip/wheels/a9/7a/45/434ba7d04afbfcc5c12195682413f53ecd4580dfd62905a672
  Created wheel for memoized-property: filename=memoized_property-1.0.3-py2.py3-none-any.whl size=41

In [ ]:
import os
import sys

from urllib.request import urlretrieve

import Bio
from Bio import SeqIO, SearchIO, Entrez
from Bio.Seq import Seq
from Bio.SeqUtils import GC
from Bio.Blast import NCBIWWW
from Bio.Data import CodonTable

print("Python version:", sys.version_info)
print("Biopython version:", Bio.__version__)

Python version: sys.version_info(major=3, minor=7, micro=12, releaselevel='final', serial=0)
Biopython version: 1.79


In [ ]:
from google.colab import drive

#make sure to add the shared group directory as a shortcut in drive 
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
# Set data file location
# If you are running notebooks on your laptop, change this to the directory
# where you put downloaded files

from pathlib import Path

literature = Path("/content/drive/MyDrive/ecbm-e4060-covid-interactions/DATA/Literature")
hackathon = Path("/content/drive/MyDrive/ecbm-e4060-covid-interactions/DATA/Hackathon")

In [ ]:
#load packages 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu
import seaborn as sns
from Bio import SeqIO

## load PWMs and RBP data that we used in hackathon re-analysis 

In [ ]:
#1. load PWMs that we initially downloaded to replicate the hackathon analysis 

rbps = pd.read_csv(hackathon / "RBPs_used_in_hackathon.txt", sep=">")
print(rbps.head())
print(rbps.shape)
len(rbps["Gene_name"].unique()) #102 proteins with PWMs

  Gene_name          Gene_id Mutated  ...    Pubmed Family Matrix_id
0      A1CF  ENSG00000148584      no  ...  23846655    RRM  M001_0.6
1    ANKHD1  ENSG00000131503      no  ...  23846655     KH  M002_0.6
2     CELF1  ENSG00000149187      no  ...  16938098    RRM       s86
3     CELF2  ENSG00000048740      no  ...  15657417    RRM        s4
4     CELF2  ENSG00000048740      no  ...  15657417    RRM        s5

[5 rows x 9 columns]
(209, 9)


102

In [ ]:
input_file = hackathon / "pwm.txt"
for seq_record in SeqIO.parse(input_file, "fasta"):
  print(seq_record)

In [ ]:
#as input to our model we more likely want a matrix where each row 
#is a protein that binds RNA and associated columns contain motifs and other
#properties associated with the protein 

## load predictions made in hackathon for RNA binding proteins related specifically to COVID-19

In [ ]:
#2. load in predictions of proteins from hackathon that bind covid-19 RNA 
predictions_genome_covid = pd.read_csv(hackathon / "sars_2_genomewide_hits.txt", sep=";")
predictions_genome_covid

,Gene_name,strand,N,mean_count,sd_count,z,pval,qval
0,HNRNPL,+,632,387.5538,20.243979,12.075008,7.157531e-34,7.085956e-32
1,FUS,+,140,61.4582,7.443073,10.552335,2.477497e-26,1.226361e-24
2,MBNL1,+,682,402.2202,29.912798,9.353181,4.252541e-21,1.403339e-19
3,SRSF1,+,335,221.2350,15.190995,7.488976,3.470646e-14,8.589849e-13
4,RBMY1A1,+,107,55.8790,7.316165,6.987404,1.400092e-12,2.772182e-11
5,ZFP36,+,609,469.4026,20.582304,6.782399,5.909821e-12,9.751205e-11
6,SRSF10,+,88,42.4038,7.098959,6.422941,6.683299e-11,9.452095e-10
7,PTBP1,+,3151,2828.7736,54.425648,5.920488,1.604938e-09,1.986111e-08
8,YBX2,+,51,24.1720,4.919640,5.453245,2.472947e-08,2.720242e-07
9,SRSF3,+,74,38.8242,6.483896,5.425103,2.896061e-08,2.867100e-07


## load newly published data that was not used in hackathon analysis and prepare as input for our machine learning analysis 

In [ ]:
#3. load new data obtained from recent publications related to RNA and proteins
#that are associated with COVID-19 viral infection 

In [ ]:
#The SARS-CoV-2 RNA–protein interactome in infected human cells, December 2020
#supplementary table 1 (Proteins detected by quantitative mass spectrometry in 
#SARS-CoV-2 RNA and RMRP RNA antisense purifications in infected human cells.)

st1 = literature / "41564_2020_846_MOESM2_ESM-1.xlsx"
st1_pd = pd.read_excel(st1, index_col=0, skiprows=2)
st1_pd[st1_pd['adj.P.Val.SCoV2.over.RMRP'] < 0.05]

,adj.P.Val.SCoV2.over.RMRP,Log.P.Value.SCoV2.over.RMRP,logFC.SCoV2.over.RMRP,P.Value.SCoV2.over.RMRP,RMRP rep2 ratio,RMRP rep1 ratio,SCoV2 rep2 ratio,SCoV2 rep1 ratio,accession_number,geneSymbol,numSpectraProteinObserved,numPepsUnique,numPepsUniqueSubgroupSpecificCI,accession_numbers,species,percentCoverage,groupNum,subgroupNum,scoreUnique,scoreUniqueSubgroupSpecificCI,totalIntensity,entry_name
id,,,,,,,,,,,,,,,,,,,,,,
NP_001120665.1,0.000111,60.516031,3.939615,8.879672e-07,NaN,-1.783388,2.424939,1.887515,NP_001120665.1,CNBP,1,9,1,NP_001120665.1|NP_001120667.1,HOMO SAPIENS,52.2,79,79.2,123.53,14.78,1330000,cellular nucleic acid-binding protein isoform...
YP_009742613.1,0.000290,54.270089,3.407615,3.741029e-06,NaN,-0.205388,3.379939,3.024515,YP_009742613.1,nsp6,1,1,1,YP_009742613.1,SARSCOV2_NC_045512.2,2.4,613,613.1,14.80,14.80,17500000,mat_peptide:nsp6:ORF1a_polyprotein:ORF1ab:GN=...
NP_001129125.1,0.002919,40.563732,2.976258,8.782675e-05,-1.431673,-0.306388,2.542939,1.671515,NP_001129125.1,PABPC4,13,16,8,NP_001129125.1|NP_001129126.1|NP_003810.1|NP_0...,HOMO SAPIENS,24.8,18,18.2,256.13,128.11,1810000000,polyadenylate-binding protein 4 isoform 1 GN=...
YP_009725310.1,0.002303,41.804432,2.620615,6.600195e-05,NaN,-0.079388,2.759939,2.322515,YP_009725310.1,nsp15,1,1,1,YP_009725310.1,SARSCOV2_NC_045512.2,2.6,617,617.1,14.66,14.66,3760000,mat_peptide:endoRNAse:ORF1b_polyprotein:ORF1a...
NP_001138898.1,0.002303,42.016585,2.586615,6.285524e-05,NaN,0.004612,2.700939,2.481515,NP_001138898.1,YBX3,1,13,1,NP_001138898.1,HOMO SAPIENS,55.7,23,23.3,228.46,15.56,9240000,Y-box-binding protein 3 isoform b GN=YBX3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NP_001032827.1,0.000111,58.965961,-4.115242,1.268831e-06,2.200327,0.923612,-2.658061,-2.448485,NP_001032827.1,NPM1,15,8,8,NP_001032827.1|NP_001341935.1|NP_001341938.1|N...,HOMO SAPIENS,30.8,101,101.1,106.93,106.93,3830000000,nucleophosmin isoform 3 GN=NPM1
NP_001306038.1,0.000099,62.461566,-4.317742,5.673400e-07,2.261327,1.158612,-2.931061,-2.284485,NP_001306038.1,NEPRO,4,4,4,NP_001306038.1|NP_001306039.1|NP_001306040.1|N...,HOMO SAPIENS,7.0,283,283.1,36.68,36.68,29500000,nucleolus and neural progenitor protein isofo...
NP_006618.1,0.000002,82.423404,-4.691242,5.723472e-09,2.525327,1.620612,-2.704061,-2.532485,NP_006618.1,POP4,6,3,3,NP_006618.1,HOMO SAPIENS,13.6,324,324.1,31.54,31.54,43600000,ribonuclease P protein subunit p29 GN=POP4


In [ ]:
#SARS-CoV-2 uses a multipronged strategy to impede host protein synthesis, Nature, May 2021

#RNA-seq results for covid infection (RPKM values)
#gene symbols and identifiers in the column "names", cluster division 
#(as shown in extended data figure 4a) shown in column "cluster", and RPKM 
#expression levels in mRNA-seq (mrna) and Ribo-seq (fp) libraries along infection

rnaseq = literature / "41586_2021_3610_MOESM3_ESM.csv"
rnaseq = pd.read_csv(rnaseq, skiprows=3)
rnaseq.head()

#clean up gene symbol column 

,names,cluster,mrna_uninf_rep1,mrna_uninf_rep2,mrna_03hr,mrna_05hr,mrna_08hr_rep1,mrna_08hr_rep2,fp_uninf_rep1,fp_uninf_rep2,fp_03hr,fp_05hr,fp_08hr,fp_08hr_rep1,fp_08hr_rep2
0,AASS|uc003vkb.3,A,3.082664,6.232156,5.644265,4.012422,9.116365,8.580547,7.523266,6.985052,3.298031,1.710406,6.155001,4.601098,7.708903
1,ADM|uc001mil.1,A,13.406347,9.561548,8.433032,18.658462,30.748639,28.761611,32.000331,37.138780,28.251279,38.093971,37.950914,37.558749,38.343079
2,AHR|uc011jxz.1,C,67.753294,144.828508,182.719179,123.486065,180.125829,146.745706,362.538979,271.598288,709.898946,245.027362,256.478270,227.116929,285.839611
3,AMOTL2|uc003eqg.1,A,79.061076,65.232246,42.437984,117.869569,183.534598,162.564791,99.153743,96.935743,109.120840,74.770744,95.728853,90.194263,101.263444
4,ANKRD12|uc002knv.3,C,3.395453,8.782348,11.365398,6.412294,13.640186,8.714097,4.179031,1.940032,7.280475,1.190556,3.040658,3.633276,2.448039


In [ ]:
#Global analysis of protein-RNA interactions in SARS-CoV-2-infected cells 
#reveals key regulators of infection, Molecular Cell July, 2021

#Table S1. Analysis of RBP dynamics in SARS-CoV-2-infected cells by cRIC, 
#related to Figure 1. cRIC datasets were generated at 8 and 24 hpi.

cRIC_RBPs = literature / "1-s2.0-S109727652100407X-mmc2.xlsx"
cRIC_RBPs_8hours = pd.read_excel(cRIC_RBPs, sheet_name="SARS-Cov2_cRIC_(Inf vs M) 8hpi")
cRIC_RBPs_24hours = pd.read_excel(cRIC_RBPs, sheet_name="SARS-Cov2_cRIC_(Inf vs M) 24hpi")

cRIC_RBPs_8hours.head()
cRIC_RBPs_24hours.head()
cRIC_RBPs_24hours[cRIC_RBPs_24hours['adj.P.Val'] < 0.05]

,Unique.protein.ID,Gene.name,logFC,adj.P.Val,Majority.protein.IDs,Gene.names
0,A0AV96,RBM47,0.774331,0.014069,A0AV96,RBM47
2,A5YKK6-3,CNOT1,-2.832246,0.011623,A5YKK6-3;A5YKK6-2;A5YKK6;A5YKK6-4,CNOT1
4,O00178,GTPBP1,3.376055,0.004177,O00178,GTPBP1
7,O00411,POLRMT,-2.134347,0.024285,O00411,POLRMT
8,O00422,SAP18,1.138107,0.029449,O00422,SAP18
...,...,...,...,...,...,...
804,sp|P0DTC1|R1A_SARS2,sp|P0DTC1|R1A_SARS2,11.610643,0.000002,sp|P0DTC1|R1A_SARS2;sp|P0DTD1|R1AB_SARS2,NaN
805,sp|P0DTC2|SPIKE_SARS2,sp|P0DTC2|SPIKE_SARS2,8.664191,0.000017,sp|P0DTC2|SPIKE_SARS2,NaN
806,sp|P0DTC5|VME1_SARS2,sp|P0DTC5|VME1_SARS2,8.651646,0.000013,sp|P0DTC5|VME1_SARS2,NaN
807,sp|P0DTC9|NCAP_SARS2,sp|P0DTC9|NCAP_SARS2,13.196982,0.000017,sp|P0DTC9|NCAP_SARS2,NaN


In [ ]:
###get protein amino acid sequences for all the RBPs we are evaluating 

#these are just spike proteins
seq_id_of_interest = [
    'NP_001120665.1',
    'NP_001129125.1'
]

# Fetch the 12 sequences of interest of S protein and save them in a fasta file

from Bio import Entrez
Entrez.email = "ki2255@cumc.columbia.edu"

#retrieve S protein sequences 
handle = Entrez.efetch(db="protein", id=seq_id_of_interest, rettype="fasta")

#save sequences to fasta file
fasta_file = "covid.fasta"
with open(fasta_file, "w") as fo:
    fo.write(handle.read())

#check that everything worked
!cat covid.fasta

>NP_001120665.1 CCHC-type zinc finger nucleic acid binding protein isoform 2 [Homo sapiens]
MSSNECFKCGRSGHWARECPTGGGRGRGMRSRGRGGFTSDRGFQFVSSSLPDICYRCGESGHLAKDCDLQ
EDEACYNCGRGGHIAKDCKEPKREREQCCYNCGKPGHLARDCDHADEQKCYSCGEFGHIQKDCTKVKCYR
CGETGHVAINCSKTSEVNCYRCGESGHLARECTIEATA

>NP_001129125.1 polyadenylate-binding protein 4 isoform 1 [Homo sapiens]
MNAAASSYPMASLYVGDLHSDVTEAMLYEKFSPAGPVLSIRVCRDMITRRSLGYAYVNFQQPADAERALD
TMNFDVIKGKPIRIMWSQRDPSLRKSGVGNVFIKNLDKSIDNKALYDTFSAFGNILSCKVVCDENGSKGY
AFVHFETQEAADKAIEKMNGMLLNDRKVFVGRFKSRKEREAELGAKAKEFTNVYIKNFGEEVDDESLKEL
FSQFGKTLSVKVMRDPNGKSKGFGFVSYEKHEDANKAVEEMNGKEISGKIIFVGRAQKKVERQAELKRKF
EQLKQERISRYQGVNLYIKNLDDTIDDEKLRKEFSPFGSITSAKVMLEDGRSKGFGFVCFSSPEEATKAV
TEMNGRIVGSKPLYVALAQRKEERKAHLTNQYMQRVAGMRALPANAILNQFQPAAGGYFVPAVPQAQGRP
PYYTPNQLAQMRPNPRWQQGGRPQGFQGMPSAIRQSGPRPTLRHLAPTGNAPASRGLPTTTQRVGSECPD
RLAMDFGGAGAAQQGLTDSCQSGGVPTAVQNLAPRAAVAAAAPRAVAPYKYASSVRSPHPAIQPLQAPQP
AVHVQGQEPLTASMLAAAPPQEQKQMLGERLFPLIQTMHSNLAGKITGMLLEIDNSELLHMLESPESLRS
KVDEAVAVLQAHHA

In [ ]:
###get all covid sequences that are available 

#these are just spike proteins
seq_id_of_interest = [
    'MW590397',
'MW592612',
'MW592636',
'MZ500802',
'MZ500903',
]

# Fetch the 12 sequences of interest of S protein and save them in a fasta file

from Bio import Entrez
Entrez.email = "ki2255@cumc.columbia.edu"

#retrieve S protein sequences 
handle = Entrez.efetch(db="nucleotide", id=seq_id_of_interest, rettype="fasta")

#save sequences to fasta file
fasta_file = "covid.fasta"
with open(fasta_file, "w") as fo:
    fo.write(handle.read())

#check that everything worked
!cat covid.fasta

In [ ]:
record = Entrez.read(Entrez.einfo())
record['DbList']

['pubmed', 'protein', 'nuccore', 'ipg', 'nucleotide', 'structure', 'genome', 'annotinfo', 'assembly', 'bioproject', 'biosample', 'blastdbinfo', 'books', 'cdd', 'clinvar', 'gap', 'gapplus', 'grasp', 'dbvar', 'gene', 'gds', 'geoprofiles', 'homologene', 'medgen', 'mesh', 'ncbisearch', 'nlmcatalog', 'omim', 'orgtrack', 'pmc', 'popset', 'proteinclusters', 'pcassay', 'protfam', 'biosystems', 'pccompound', 'pcsubstance', 'seqannot', 'snp', 'sra', 'taxonomy', 'biocollections', 'gtr']